In [ ]:
const wrapInHtml = (content) => {
  if (content == null) {
    console.error("Received null or undefined content in wrapInHtml");
    return "<html><body><p>Error: No content available.</p></body></html>";
  }

  const processMarkdown = (text) => {
    if (typeof text !== 'string') {
      console.error("Non-string content passed to processMarkdown:", text);
      return "";
    }

    const processList = (list) => {
      const lines = list.split('\n');
      const result = { html: '', lastIndentLevel: -1 };

      const processListItem = (line, result) => {
        const match = line.match(/^(\s*)(\d+\.|-)\s+(.+)/);
        if (!match) return result;

        const [, indent, bullet, content] = match;
        const indentLevel = indent.length / 2;
        const listType = indentLevel === 0 && bullet !== '-' ? 'ol' : 'ul';

        // Close lists if we're moving to a lower indent level
        while (result.lastIndentLevel >= indentLevel) {
          result.html += `</li></${result.lastIndentLevel === 0 ? 'ol' : 'ul'}>`;
          result.lastIndentLevel--;
        }

        // Open new list if needed
        if (result.lastIndentLevel < indentLevel) {
          result.html += `<${listType}>`;
          result.lastIndentLevel = indentLevel;
        } else if (result.lastIndentLevel === indentLevel) {
          result.html += '</li>';
        }

        result.html += `<li>${content}`;
        return result;
      };

      lines.forEach(line => {
        result = processListItem(line, result);
      });

      // Close any remaining open lists
      while (result.lastIndentLevel >= 0) {
        result.html += `</li></${result.lastIndentLevel === 0 ? 'ol' : 'ul'}>`;
        result.lastIndentLevel--;
      }

      return result.html;
    };

    // Process block elements
    text = text
      .replace(/^(#{1,6})\s+(.*?)$/gm, (match, hashes, content) => {
        const level = hashes.length;
        return `<h${level}>${content}</h${level}>`;
      })
      .replace(/(?:(?:\n|^)(?:\s*)(?:\d+\.|-)\s+(.+)(?:\n|$))+/g, processList)
      .replace(/(\n|^)(?!<\/?(ul|ol|li|h[1-6]|p|a|code|pre|strong|em|hr))(.+)/g, '$1<p>$3</p>');

    // Process inline elements
    text = text
      .replace(/\[([^\]]+)\]\(([^)]+)\)/g, '<a href="$2">$1</a>')
      .replace(/\*\*(.*?)\*\*/g, '<strong>$1</strong>')
      .replace(/\*(.*?)\*/g, '<em>$1</em>')
      .replace(/```(\w+)?\n([\s\S]*?)```/g, '<pre><code>$2</code></pre>')
      .replace(/`([^`]+)`/g, '<code>$1</code>')
      .replace(/^---$/gm, '<hr>');

    return text;
  };

  try {
    const processedContent = processMarkdown(content);
    return `
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <style>
        body {
            font-family: Arial, sans-serif;
            font-size: 11pt;
            color: #000;
            max-width: 800px;
            margin: 0 auto;
            padding: 20px;
        }
        h1, h2, h3, h4, h5, h6 {
            margin-top: 1em;
            margin-bottom: 0.5em;
        }
        p {
            margin-top: 0;
            margin-bottom: 1em;
        }
        pre {
            background-color: #f4f4f4;
            padding: 10px;
            border-radius: 4px;
            overflow-x: auto;
        }
        code {
            background-color: #f4f4f4;
            padding: 2px 4px;
            border-radius: 2px;
            font-family: monospace;
        }
        ul, ol {
            padding-left: 30px;
            margin-bottom: 1em;
        }
        li {
            margin-bottom: 0.5em;
        }
        hr {
            border: none;
            border-top: 1px solid #ccc;
            margin: 20px 0;
        }
        a {
            color: #0066cc;
            text-decoration: none;
        }
        a:hover {
            text-decoration: underline;
        }
    </style>
</head>
<body>
    <div class="email-content">${processedContent}</div>
</body>
</html>
    `;
  } catch (error) {
    console.error("Error in wrapInHtml:", error);
    return "<html><body><p>Error processing content.</p></body></html>";
  }
};